In [5]:
#installing necessary libraries
!pip install requests
!pip install beautifulsoup4
!pip install nltk
!pip install openpyxl
!pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Now Importing the libraries

import requests
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# Loading input data
input_df = pd.read_excel('Input.xlsx')

# Loading stop words
stop_words_files = [
    'StopWords_Auditor.txt', 'StopWords_Currencies.txt', 
    'StopWords_DatesandNumbers.txt', 'StopWords_Generic.txt', 
    'StopWords_GenericLong.txt', 'StopWords_Geographic.txt', 
    'StopWords_Names.txt'
]

stop_words = set()
for file in stop_words_files:
    with open(file, 'r') as f:
        stop_words.update(f.read().splitlines())

# Load positive and negative words
with open('positive-words.txt', 'r') as f:
    positive_words = set(f.read().splitlines())

with open('negative-words.txt', 'r') as f:
    negative_words = set(f.read().splitlines())

# Filtering out stop words from positive and negative words
positive_words = positive_words - stop_words
negative_words = negative_words - stop_words


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hitej\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hitej\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# defining the function to extract urls
def extract_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('title').text.strip()
        paragraphs = soup.find_all('p')
        article_text = ' '.join([p.get_text() for p in paragraphs])
        return title, article_text
    except Exception as e:
        print(f"Error extracting {url}: {e}")
        return None, None

for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    title, text = extract_text(url)
    if text:
        with open(f'{url_id}.txt', 'w', encoding='utf-8') as f:
            f.write(f'{title}\n{text}')


In [7]:
# Analysis Part

def analyze_text(text):
    # Clean text
    tokens = word_tokenize(text)
    cleaned_tokens = [word for word in tokens if word.lower() not in stop_words and word.isalpha()]
    
    # Positive and Negative Scores
    positive_score = sum(1 for word in cleaned_tokens if word.lower() in positive_words)
    negative_score = sum(1 for word in cleaned_tokens if word.lower() in negative_words)
    
    # Polarity Score
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    
    # Subjectivity Score
    subjectivity_score = (positive_score + negative_score) / (len(cleaned_tokens) + 0.000001)
    
    # Readability Analysis
    sentences = sent_tokenize(text)
    avg_sentence_length = len(cleaned_tokens) / len(sentences)
    complex_words = [word for word in cleaned_tokens if len(word) > 2]
    percentage_complex_words = len(complex_words) / len(cleaned_tokens)
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    
    # Additional Metrics
    avg_words_per_sentence = len(cleaned_tokens) / len(sentences)
    complex_word_count = len(complex_words)
    word_count = len(cleaned_tokens)
    syllable_count = sum([sum([1 for char in word if char in 'aeiouAEIOU']) for word in cleaned_tokens])
    syllables_per_word = syllable_count / len(cleaned_tokens)
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))
    avg_word_length = sum(len(word) for word in cleaned_tokens) / len(cleaned_tokens)
    
    return [
        positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, 
        percentage_complex_words, fog_index, avg_words_per_sentence, complex_word_count, word_count, 
        syllables_per_word, personal_pronouns, avg_word_length
    ]

results = []
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    with open(f'{url_id}.txt', 'r', encoding='utf-8') as f:
        text = f.read()
        analysis_results = analyze_text(text)
        results.append([url_id] + analysis_results)

columns = [
    'URL_ID', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
    'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 
    'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 
    'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'
]
output_df = pd.DataFrame(results, columns=columns)
output_df.to_excel('Output Data Structure.xlsx', index=False)
